# 联合网格细胞 Conjuctive grid cells  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RatInABox-Lab/RatInABox/blob/main/demos/conjunctive_gridcells_example.ipynb)


In this demo we'll show a very simple way one could model conjuctive grid cells (Sargolini, 2006) by non-linearly combining `GridCells` and `HeadDirectionCells` using `FeedForwardLayer`.

在这个演示中，我们将展示一种非常简单的方法，通过使用 `FeedForwardLayer` 非线性组合 `GridCells` 和 `HeadDirectionCells` 来建模联合网格细胞（Sargolini, 2006）。


Here's a schematic of the simulation we're going to build: 

这是我们将要构建的模拟示意图：

* Grid cells are __spatially selective__ but not head direction selective
* Head direction cells are __head direction selective__ but not spatially selective
* Conjunctive grid cells are __jointly spatially and head direction selective__

* 网格细胞具有**空间选择性**但没有头部方向选择性
* 头部方向细胞具有**头部方向选择性**但没有空间选择性
* 联合网格细胞**同时具有空间和头部方向选择性**

<img src="../.images/demos/conjunctive_grid_cells.png" width="500" align="center">

In [ ]:
# Install dependencies 
!pip install ratinabox

#Import ratinabox 
import ratinabox
from ratinabox.Environment import Environment
from ratinabox.Agent import Agent
from ratinabox.Neurons import GridCells, HeadDirectionCells, FeedForwardLayer

#stylize plots and set figure directory for saving 
ratinabox.stylize_plots(); ratinabox.autosave_plots=True; ratinabox.figure_directory="../figures/"

#...and other dependencies
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt

#Some global variables
N_cells = 3

### 创建 `GridCells` 和 `HeadDirectionCells` 并绘制它们的空间和头部方向选择性

> Make `GridCells` and `HeadDirectionCells` and plot their spatial and head direction selectivity


In [ ]:
Env = Environment()
Ag = Agent(Env)
GCs = GridCells(Ag, params = {
    'n':N_cells,
    'gridscale':[0.2,0.4,0.6],
    'name':'GridCells',
    'color':'C1'})
HDCs = HeadDirectionCells(Ag, params={
    'n':N_cells,
    'name':'HeadDirectionCells',
    'color':'C5'})

Plot the spatial and head direction selectivity of the `GridCells` and `HeadDirectionCells` we're going to use to make the conjunctive grid cells.

绘制我们将用于创建联合网格细胞的 `GridCells` 和 `HeadDirectionCells` 的空间和头部方向选择性。

* `Neurons.plot_rate_map(method="groundtruth_headdirectionaveraged")` calcultes the rate maps at all head directions, _averages over head direction_ and then plots the spatial rate map. 
* `Neurons.plot_angular_rate_map()` calculates the rate maps at all head directions and then _averages over space_ and plots the head direction rate map in polar coordinates.

* `Neurons.plot_rate_map(method="groundtruth_headdirectionaveraged")` 计算所有头部方向的速率图，_对头部方向进行平均_，然后绘制空间速率图。
* `Neurons.plot_angular_rate_map()` 计算所有头部方向的速率图，然后_对空间进行平均_，并以极坐标形式绘制头部方向速率图。

In [ ]:
fig, ax = GCs.plot_rate_map(method="groundtruth_headdirectionaveraged",colorbar=False)
fig.suptitle("Spatial selectivity of grid cells")
fig, ax = GCs.plot_angular_rate_map()
fig.suptitle("Head direction selectivity of grid cells (there is none)")

In [ ]:
fig, ax = HDCs.plot_rate_map(method="groundtruth_headdirectionaveraged",colorbar=False)
fig.suptitle("Spatial selectivity of head direction cells (there is none)")
fig, ax = HDCs.plot_angular_rate_map()
# fig.suptitle("Head direction selectivity of head direction cells")

## 创建联合网格细胞

> Make conjunctive grid cells

The way we'll do this is we'll make a `FeedForwardLayer` which takes as input the `GridCells` and `HeadDirectionCells` and then non-linearly combines them to make conjunctive grid cells.

我们的做法是创建一个 `FeedForwardLayer`，它将 `GridCells` 和 `HeadDirectionCells` 作为输入，然后非线性地组合它们以创建联合网格细胞。

$$F(x,\phi_h) = \sigma\big( \mathsf{W}_g \cdot F_{g}(x) + \mathsf{W}_h \cdot F_{h}(\phi_{h})\big)$$

where $F_{g}(x)$ is the grid cell activity, $F_{h}(\phi_{h})$ is the head direction cell activity, $\mathsf{W}_g$ and $\mathsf{W}_h$ are the weight matrices for the grid cells and head direction cells respectively, and $\sigma$ is a non-linearity.

其中 $F_{g}(x)$ 是网格细胞活动，$F_{h}(\phi_{h})$ 是头部方向细胞活动，$\mathsf{W}_g$ 和 $\mathsf{W}_h$ 分别是网格细胞和头部方向细胞的权重矩阵，$\sigma$ 是非线性函数。

For each input we'll make the weight matrix the identity: $[\mathsf{W}_g]_{ij} = [\mathsf{W}_h]_{ij} = \delta_{ij}$. The non-linearity will be a relu function thresholded at 1.0, $\sigma(x) = \max(0,x-1)$.  Since the maximum firing rate of the `GridCells` and `HeadDirectionCells` is 1.0, _only_ if the grid cell is active _and_ the head direction cell are active together will the total conjunctive grid cell input overcome the threshold and fire. 

对于每个输入，我们将权重矩阵设置为单位矩阵：$[\mathsf{W}_g]_{ij} = [\mathsf{W}_h]_{ij} = \delta_{ij}$。非线性函数将是阈值为 1.0 的 relu 函数，$\sigma(x) = \max(0,x-1)$。由于 `GridCells` 和 `HeadDirectionCells` 的最大发放率为 1.0，_只有_当网格细胞活跃_并且_头部方向细胞同时活跃时，联合网格细胞的总输入才会超过阈值并发放。

Here's how you'd write this with RatInABox...

以下是如何使用 RatInABox 编写这个功能...


In [ ]:
CGCs = FeedForwardLayer(Ag, params={
    'n':N_cells,
    'input_layers':[GCs,HDCs],
    'activation_function':{'activation':'relu',
                           'threshold':1},
    'color':'C0'}
    )

#It's also possible to write your own activation function and have biases as well as weights. Feel free to experiment (read the documatation/paper for full info)!

CGCs.inputs['GridCells']['w'] = np.identity(N_cells)
CGCs.inputs['HeadDirectionCells']['w'] = np.identity(N_cells)

In [ ]:
fig, ax = CGCs.plot_rate_map(method="groundtruth_headdirectionaveraged",colorbar=False)
fig.suptitle("Spatial selectivity of conjunctive grid cells (like grid cells)")
fig, ax = CGCs.plot_angular_rate_map()
fig.suptitle("Head direction selectivity of conjunctive grid cells (like head direction cells)")

### 模拟它们以确保行为符合预期

> Simulate them to make sure they behave as expected

Final lets simulate them for two minutes and plot the firing rate data. We'll also plot the rat's trajectory and head direction to make sure the firing rate data makes sense.

最后，让我们对它们进行两分钟的模拟并绘制发放率数据。我们还将绘制大鼠的运动轨迹和头部方向，以确保发放率数据是合理的。

We'll make the rat move under circular motion. This should be clearly visible in the head direction cell data as cells periodincal fire as the rat's head direction changes. The conjunctive grid cells should fire a bit like the grid cells do but modulated by the activity of their respective head direction cell.

我们将让大鼠进行圆周运动。这应该在头部方向细胞数据中清晰可见，因为当大鼠的头部方向改变时，细胞会周期性地发放。联合网格细胞的发放应该类似于网格细胞，但会受到其对应的头部方向细胞活动的调制。

In [ ]:
# 模拟大鼠圆周运动 120 秒
while Ag.t < 120:
    r = Ag.pos - Env.centre  # 计算相对于中心的位置向量
    drift = np.array([-r[1],r[0]])  # 切向漂移速度（垂直于半径方向，实现圆周运动）
    drift = 0.2 * drift / np.linalg.norm(drift)  # 归一化并设置速度大小
    Ag.update(drift_velocity=drift,drift_to_random_strength_ratio=4)  # 更新智能体位置（主要沿圆周，加入少量随机性）
    GCs.update()  # 更新网格细胞活动
    HDCs.update()  # 更新头部方向细胞活动
    CGCs.update()  # 更新联合网格细胞活动

# 绘制结果
Ag.plot_trajectory(color='changing',colorbar=True)  # 绘制运动轨迹（颜色表示时间）
GCs.plot_rate_timeseries()  # 绘制网格细胞发放率时间序列
HDCs.plot_rate_timeseries()  # 绘制头部方向细胞发放率时间序列
CGCs.plot_rate_timeseries()  # 绘制联合网格细胞发放率时间序列